# Combine Movie Functions

In [16]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from wikipedia import wikipedia
from oscar import oscars
from boxofficemojo import boxofficemojo
from rottentomatoes import rottentomatoes
from combine import combine


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2018-12-28 14:20:42.095145


# Get/Parse/Merge/Process Rotten Tomatoes Data

In [17]:
comb = combine()
comb.setOscarData(oscar=oscars(wikipedia()))
comb.setRottenTomatoesData(rottentomatoes=rottentomatoes())
comb.setBoxOfficeMojoData(boxofficemojo=boxofficemojo())

In [18]:
comb.getData()

Found 36 Years of BoxOfficeMojo Data
Found 84 Years of Oscar Data
Found 93 Years of RottenTomatoes Data
Found Data Between 1915 and 2017


In [19]:
comb.mergeMovies(oscarVal=10, rottentomatoesVal=90, boxofficeVal=1e7, debug=True)

Found 7560 movies
There are 7180 final movies
Saving data to /Users/tgadfort/Documents/code/movies/combine/results/movies.yaml
Saved data to /Users/tgadfort/Documents/code/movies/combine/results/movies.yaml
  --> This file is 2.8MB.


In [20]:
comb.movies

OrderedDict([('The Birth Of A Nation [1915]', 'Rotten Tomatoes'),
             ('The Cabinet Of Dr. Caligari (Das Cabinet des Dr. Caligari) [1920]',
              'Rotten Tomatoes'),
             ('Nosferatu, A Symphony Of Horror (Nosferatu, eine Symphonie des Grauens) (Nosferatu The Vampire) [1922]',
              'Rotten Tomatoes'),
             ('The Gold Rush [1925]', 'Rotten Tomatoes'),
             ('Battleship Potemkin [1925]', 'Rotten Tomatoes'),
             ('Phantom Of The Opera [1925]', 'Rotten Tomatoes'),
             ('Metropolis [1927]', 'Rotten Tomatoes'),
             ('Sunrise A Song Of Two Humans [1927]', 'Rotten Tomatoes'),
             ('The General [1927]', 'Rotten Tomatoes'),
             ("The Passion Of Joan Of Arc (La Passion de Jeanne d'Arc) [1928]",
              'Rotten Tomatoes'),
             ('All Quiet On The Western Front [1930]', 'Rotten Tomatoes'),
             ('Frankenstein [1931]', 'Rotten Tomatoes'),
             ('M [1931]', 'Rotten Tomatoes'),
